<a href="https://colab.research.google.com/github/JafarunnishaShaik/Fraud-Transaction-Detection/blob/main/Fraud_Transaction_Detection_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [38]:
#loading the dataset to a pandas DataFrame
credit_card_data = pd.read_pickle('/content/2018-04-01.pkl')

In [39]:
#first 5 rows of the dataset
credit_card_data.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
0,0,2018-04-01 00:00:31,596,3156,57.16,31,0,0,0
1,1,2018-04-01 00:02:10,4961,3412,81.51,130,0,0,0
2,2,2018-04-01 00:07:56,2,1365,146.00,476,0,0,0
3,3,2018-04-01 00:09:29,4128,8737,64.49,569,0,0,0
4,4,2018-04-01 00:10:34,927,9906,50.99,634,0,0,0


In [40]:
credit_card_data.tail()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
9483,9483,2018-04-01 23:56:50,3289,6699,48.69,86210,0,0,0
9484,9484,2018-04-01 23:58:14,3230,6664,85.97,86294,0,0,0
9485,9485,2018-04-01 23:58:31,296,3702,120.88,86311,0,0,0
9486,9486,2018-04-01 23:59:28,2557,146,8.02,86368,0,0,0
9487,9487,2018-04-01 23:59:51,554,8248,19.20,86391,0,0,0


In [41]:
#data set informations
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9488 entries, 0 to 9487
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TRANSACTION_ID     9488 non-null   int64         
 1   TX_DATETIME        9488 non-null   datetime64[ns]
 2   CUSTOMER_ID        9488 non-null   object        
 3   TERMINAL_ID        9488 non-null   object        
 4   TX_AMOUNT          9488 non-null   float64       
 5   TX_TIME_SECONDS    9488 non-null   object        
 6   TX_TIME_DAYS       9488 non-null   object        
 7   TX_FRAUD           9488 non-null   int64         
 8   TX_FRAUD_SCENARIO  9488 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 999.3+ KB


In [42]:
#checking the number of missing values in each column
credit_card_data.isnull().sum()

,0
TRANSACTION_ID,0
TX_DATETIME,0
CUSTOMER_ID,0
TERMINAL_ID,0
TX_AMOUNT,0
TX_TIME_SECONDS,0
TX_TIME_DAYS,0
TX_FRAUD,0
TX_FRAUD_SCENARIO,0


In [43]:
#distribution of legit transactions & fradulent transactions
credit_card_data['TX_FRAUD'].value_counts()

,count
TX_FRAUD,
0,9485
1,3


This data set highly unblanced

0----->Normal Transaction
1----->Fraudulent transaction

In [44]:
#Separating the data for analysis
legit = credit_card_data[credit_card_data.TX_FRAUD == 0]
fraud = credit_card_data[credit_card_data.TX_FRAUD == 1]

In [45]:
print(legit.shape)
print(fraud.shape)

(9485, 9)
(3, 9)


In [46]:
#satatical measures of the data
legit.TX_AMOUNT.describe()

,TX_AMOUNT
count,9485.000000
mean,53.195756
std,39.392437
min,0.130000
25%,21.190000
50%,45.580000
75%,76.920000
max,218.120000


In [47]:
fraud.TX_AMOUNT.describe()

,TX_AMOUNT
count,3.000000
mean,224.690000
std,2.161874
min,222.260000
25%,223.835000
50%,225.410000
75%,225.905000
max,226.400000


In [48]:
#compare the values for both transactions
credit_card_data.groupby('TX_FRAUD').mean()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD_SCENARIO
TX_FRAUD,,,,,,,,
0,4743.327570,2018-04-01 12:01:30.814022144,2504.123774,5008.162256,53.195756,43290.814022,0.0,0.0
1,5288.666667,2018-04-01 12:50:31.000000000,4447.666667,6070.333333,224.690000,46231.0,0.0,1.0


Build a sample dataset Containg similar distribution of normal transactions and Fraudulent Transactions

Number of Fraudulent Transcations -->3

In [49]:
legit_sample = legit.sample(n=3)

Concatenating two DataFrames

In [50]:
new_dataset = pd.concat([legit_sample,fraud], axis = 0)

In [51]:
new_dataset.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
1479,1479,2018-04-01 06:44:56,4742,6845,6.98,24296,0,0,0
9041,9041,2018-04-01 20:26:22,1437,3458,19.28,73582,0,0,0
8002,8002,2018-04-01 17:19:41,3850,3616,89.10,62381,0,0,0
3527,3527,2018-04-01 10:17:43,3774,3059,225.41,37063,0,1,1
5790,5790,2018-04-01 13:31:48,4944,6050,222.26,48708,0,1,1


In [52]:
new_dataset.tail()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
9041,9041,2018-04-01 20:26:22,1437,3458,19.28,73582,0,0,0
8002,8002,2018-04-01 17:19:41,3850,3616,89.10,62381,0,0,0
3527,3527,2018-04-01 10:17:43,3774,3059,225.41,37063,0,1,1
5790,5790,2018-04-01 13:31:48,4944,6050,222.26,48708,0,1,1
6549,6549,2018-04-01 14:42:02,4625,9102,226.40,52922,0,1,1


In [53]:
new_dataset['TX_FRAUD'].value_counts()

,count
TX_FRAUD,
0,3
1,3


In [54]:
new_dataset.groupby('TX_FRAUD').mean()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD_SCENARIO
TX_FRAUD,,,,,,,,
0,6174.000000,2018-04-01 14:50:19.666666752,3343.0,4639.666667,38.453333,53419.666667,0.0,0.0
1,5288.666667,2018-04-01 12:50:31.000000000,4447.666667,6070.333333,224.690000,46231.0,0.0,1.0


Spliting the data into Features & Targets

In [55]:
X = new_dataset.drop(columns='TX_FRAUD',axis=1)
Y = new_dataset['TX_FRAUD']

In [56]:
print(X)

      TRANSACTION_ID         TX_DATETIME CUSTOMER_ID TERMINAL_ID  TX_AMOUNT  \
1479            1479 2018-04-01 06:44:56        4742        6845       6.98   
9041            9041 2018-04-01 20:26:22        1437        3458      19.28   
8002            8002 2018-04-01 17:19:41        3850        3616      89.10   
3527            3527 2018-04-01 10:17:43        3774        3059     225.41   
5790            5790 2018-04-01 13:31:48        4944        6050     222.26   
6549            6549 2018-04-01 14:42:02        4625        9102     226.40   

     TX_TIME_SECONDS TX_TIME_DAYS  TX_FRAUD_SCENARIO  
1479           24296            0                  0  
9041           73582            0                  0  
8002           62381            0                  0  
3527           37063            0                  1  
5790           48708            0                  1  
6549           52922            0                  1  


In [57]:
print(Y)

1479    0
9041    0
8002    0
3527    1
5790    1
6549    1
Name: TX_FRAUD, dtype: int64


Split the data into Training Data and Testing Data

In [58]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,stratify=Y, random_state=2)

In [67]:
print(X.shape, X_train.shape, X_test.shape)

(6, 8) (4, 8) (2, 8)


Model Training

Logistic Regression

In [68]:
print(X_train.dtypes)

TRANSACTION_ID                int64
TX_DATETIME          datetime64[ns]
CUSTOMER_ID                  object
TERMINAL_ID                  object
TX_AMOUNT                   float64
TX_TIME_SECONDS              object
TX_TIME_DAYS                 object
TX_FRAUD_SCENARIO             int64
dtype: object


In [74]:
# Convert TX_DATETIME to Unix timestamp (seconds since epoch)
X_train['TX_DATETIME'] = pd.to_datetime(X_train['TX_DATETIME']).astype(int) / 10**9
X_test['TX_DATETIME'] = pd.to_datetime(X_test['TX_DATETIME']).astype(int) / 10**9

In [76]:
print(X_train.columns)

Index(['TRANSACTION_ID', 'TX_DATETIME', 'TX_AMOUNT', 'TX_FRAUD_SCENARIO',
       'CUSTOMER_ID_3774', 'CUSTOMER_ID_3850', 'CUSTOMER_ID_4625',
       'TERMINAL_ID_3458', 'TERMINAL_ID_3616', 'TERMINAL_ID_9102',
       'TX_TIME_SECONDS_52922', 'TX_TIME_SECONDS_62381',
       'TX_TIME_SECONDS_73582'],
      dtype='object')


In [77]:
# Align the train and test datasets
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)


In [78]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Scale the training and test sets
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [79]:
from sklearn.linear_model import LogisticRegression

# Initialize the model
model = LogisticRegression()

# Train the model using the scaled training data
model.fit(X_train_scaled, Y_train)

LogisticRegression()

In [84]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Make predictions on the test set
Y_pred = model.predict(X_test_scaled)

# Evaluate model performance
accuracy = accuracy_score(Y_test, Y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Confusion matrix
cm = confusion_matrix(Y_test, Y_pred)
print("Confusion Matrix:\n", cm)

# Classification report
report = classification_report(Y_test, Y_pred)
print("Classification Report:\n", report)

Accuracy: 50.00%
Confusion Matrix:
 [[0 1]
 [0 1]]
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.50      1.00      0.67         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Model Evalution

Accuracy Score

In [85]:
#accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [86]:
print('Accuracy on Training data:', training_data_accuracy)

Accuracy on Training data: 0.5


In [87]:
#accuracy on text data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [88]:
print('Accuracy score on Test Data:',test_data_accuracy)

Accuracy score on Test Data: 0.5
